![title](http://neo.readthedocs.io/en/latest/_images/neologo.png)

Content taken from http://neo.readthedocs.io/en/latest/

Neo is a Python package for working with electrophysiology data in Python, together with support for reading a wide range of neurophysiology file formats, including Spike2, NeuroExplorer, AlphaOmega, Axon, Blackrock, Plexon, Tdt, Igor Pro, and support for writing to a subset of these formats plus non-proprietary formats including Kwik and HDF5.@@

The goal of Neo is to improve interoperability between Python tools for analyzing, visualizing and generating electrophysiology data, by providing a common, shared object model. In order to be as lightweight a dependency as possible, Neo is deliberately limited to represention of data, with no functions for data analysis or visualization.

Neo is used by a number of other software tools, including SpykeViewer (data analysis and visualization), Elephant (data analysis), the G-node suite (databasing), PyNN (simulations), tridesclous (spike sorting) and ephyviewer (data visualization). OpenElectrophy (data analysis and visualization) used an older version of Neo.

Neo implements a hierarchical data model well adapted to intracellular and extracellular electrophysiology and EEG data with support for multi-electrodes (for example tetrodes). Neo’s data objects build on the quantities package, which in turn builds on NumPy by adding support for physical dimensions. Thus Neo objects behave just like normal NumPy arrays, but with additional metadata, checks for dimensional consistency and automatic unit conversion.

A project with similar aims but for neuroimaging file formats is NiBabel.

To cite Neo in publications, please use:

Garcia S., Guarino D., Jaillet F., Jennings T.R., Pröpper R., Rautenberg P.L., Rodgers C., Sobolev A.,Wachtler T., Yger P. and Davison A.P. (2014) Neo: an object model for handling electrophysiology data in multiple formats. Frontiers in Neuroinformatics 8:10: doi:10.3389/fninf.2014.00010


# Installation

This notebook requires both the Neo and Elephant modules (which are not installed by default on your virtual machines). To install it, issue in the terminal:
    
    pip install neo --user
    pip install elephant --user

These packages can also be installed from their github sources (past and development versions).
To install the latest version of Neo from the Git repository:

    cd ~/sources
    git clone git://github.com/NeuralEnsemble/python-neo.git
    cd python-neo
    python setup.py install --user

To install the latest version of Elephant from the Git repository:

    cd ~/sources
    git clone git://github.com/NeuralEnsemble/elephant.git
    cd elephant
    python setup.py install --user


# Neo datatypes

![title](http://neo.readthedocs.io/en/latest/_images/base_schematic.png)

## Data objects

These objects directly represent data as arrays of numerical values with
associated metadata (units, sampling frequency, etc.).

  * `AnalogSignal`: A regular sampling of a single- or multi-channel continuous analog signal.
  * `IrregularlySampledSignal`: A non-regular sampling of a single- or multi-channel continuous analog signal.
  * `SpikeTrain`: A set of action potentials (spikes) emitted by the same unit in a period of time (with optional waveforms).
  * `Event`: An array of time points representing one or more events in the data.
  * `Epoch`: An array of time intervals representing one or more periods of time in the data.


## Container objects
-----------------

There is a simple hierarchy of containers:

  * `Segment`: A container for heterogeneous discrete or continous data sharing a common
    clock (time basis) but not necessarily the same sampling rate, start time or end time.
    A `Segment` can be considered as equivalent to a "trial", "episode", "run",
    "recording", etc., depending on the experimental context.
    May contain any of the data objects.
  * `Block`: The top-level container gathering all of the data, discrete and continuous,
    for a given recording session.
    Contains `Segment`, `Unit` and `ChannelIndex` objects.


## Grouping objects

These objects express the relationships between data items, such as which signals
were recorded on which electrodes, which spike trains were obtained from which
membrane potential signals, etc. They contain references to data objects that
cut across the simple container hierarchy.

  * `ChannelIndex`: A set of indices into `AnalogSignal` objects,
    representing logical and/or physical recording channels. This has two uses:

      1. for linking `AnalogSignal` objects recorded from the same (multi)electrode
         across several `Segment`\s.
      2. for spike sorting of extracellular signals, where spikes may be recorded on more than one
         recording channel, and the `ChannelIndex` can be used to associate each
         `Unit` with the group of recording channels from which it was obtained.

  * `Unit`: links the `SpikeTrain` objects within a `Block`,
    possibly across multiple Segments, that were emitted by the same cell.
    A `Unit` is linked to the `ChannelIndex` object from which the spikes were detected.


## NumPy compatibility

Neo data objects inherit from `Quantity`, which in turn inherits from NumPy
`ndarray`. This means that a Neo `AnalogSignal` is also a `Quantity`
and an array, giving you access to all of the methods available for those objects.

For example, you can pass a `SpikeTrain` directly to the `numpy.histogram`
function, or an `AnalogSignal` directly to the `numpy.std` function.

If you want to get a numpy.ndarray you use magnitude and rescale from quantities:

    >>> np_sig = neo_analogsignal.rescale('mV').magnitude
    >>> np_times = neo_analogsignal.times.rescale('s').magnitude

# Example

In [ ]:
# -*- coding: utf-8 -*-
"""
This is an example for reading files with neo.io
"""

import urllib
import numpy as np
import neo

url_repo = 'https://web.gin.g-node.org/NeuralEnsemble/ephy_testing_data/raw/master/'

# Plexon files
distantfile = url_repo + 'plexon/File_plexon_3.plx'
localfile = './File_plexon_3.plx'
urllib.request.urlretrieve(distantfile, localfile)

In [ ]:
# create a reader
reader = neo.io.PlexonIO(filename='File_plexon_3.plx')
# read the blocks
blks = reader.read(lazy=False)
print(blks)

In [ ]:
# access to segments
for blk in blks:
    for seg in blk.segments:
        print(seg)
        #for asig in seg.analogsignals:
        #    print(asig)
        #for st in seg.spiketrains:
        #    print(st)

In [ ]:
# plotting example
%matplotlib inline
import matplotlib.pyplot as plt
for blk in blks:
    fig = plt.figure()
    fig.suptitle(blk)
    ax0 = fig.add_subplot(2,1,1)
    ax1 = fig.add_subplot(2,1,2, sharex=ax0)
    for seg in blk.segments:
        for asig in seg.analogsignals:
            ax0.plot(asig.times.rescale('s').magnitude, 
                     asig.magnitude)
        
        for i, st in enumerate(seg.spiketrains):
            ax1.plot(st.times.rescale('s').magnitude, 
                     i*np.ones(st.size), '|')


# Neo IO

## Preamble


The Neo `io` module aims to provide an exhaustive way of loading and saving several widely used data formats in electrophysiology.
The more these heterogeneous formats are supported, the easier it will be to manipulate them as Neo objects in a similar way.
Therefore the IO set of classes propose a simple and flexible IO API that fits many format specifications.
It is not only file-oriented, it can also read/write objects from a database.

At the moment, there are 3 families of IO modules:
    1. for reading closed manufacturers' formats (Spike2, Plexon, AlphaOmega, BlackRock, Axon, ...)
    2. for reading(/writing) formats from open source tools (KlustaKwik, Elan, WinEdr, WinWcp, PyNN, ...)
    3. for reading/writing Neo structure in neutral formats (HDF5, .mat, ...) but with Neo structure inside (NeoHDF5, NeoMatlab, ...)

Combining **1** for reading and **3** for writing is a good example of use: converting your datasets
to a more standard format when you want to share/collaborate.


## Introduction

There is an intrinsic structure in the different Neo objects, that could be seen as a hierachy with cross-links. See `core`.
The highest level object is the `Block` object, which is the high level container able to encapsulate all the others.

A `Block` has therefore a list of `Segment` objects, that can, in some file formats, be accessed individually.
Depending on the file format, i.e. if it is streamable or not, the whole `Block` may need to be loaded, but sometimes
particular `Segment` objects can be accessed individually.
Within a `Segment`, the same hierarchical organisation applies.
A `Segment` embeds several objects, such as `SpikeTrain`,
`AnalogSignal`, `IrregularlySampledSignal`, `Epoch`, `Event`
(basically, all the different Neo objects).

Depending on the file format, these objects can sometimes be loaded separately, without the need to load the whole file.
If possible, a file IO therefore provides distinct methods allowing to load only particular objects that may be present in the file.
The basic idea of each IO file format is to have, as much as possible, read/write methods for the individual encapsulated objects,
and otherwise to provide a read/write method that will return the object at the highest level of hierarchy
(by default, a `Block` or a `Segment`).

The :mod:`neo.io` API is a balance between full flexibility for the user (all `read_XXX` methods are enabled)
and simple, clean and understandable code for the developer (few `read_XXX` methods are enabled).
This means that not all IOs offer the full flexibility for partial reading of data files.


## One format = one class
The basic syntax is as follows. If you want to load a file format that is implemented in a generic `MyFormatIO` class:

    >>> from neo.io import MyFormatIO
    >>> reader = MyFormatIO(filename="myfile.dat")

you can replace `MyFormatIO` by any implemented class, see `list_of_io`

## Modes

IO can be based on a single file, a directory containing files, or a database.
This is described in the `mode` attribute of the IO class:

    >>> from neo.io import MyFormatIO
    >>> print MyFormatIO.mode
    'file'


For *file* mode the *filename* keyword argument is necessary.
For *directory* mode the *dirname* keyword argument is necessary.

Ex:

    >>> reader = io.PlexonIO(filename='File_plexon_1.plx')
    >>> reader = io.TdtIO(dirname='aep_05')


## Supported objects/readable objects

To know what types of object are supported by a given IO interface:

    >>> MyFormatIO.supported_objects
    [Segment , AnalogSignal , SpikeTrain, Event, Spike]

Supported objects does not mean objects that you can read directly. For instance, many formats support `AnalogSignal`
but don't allow them to be loaded directly, rather to access the `AnalogSignal` objects, you must read a `Segment`:

    >>> seg = reader.read_segment()
    >>> print(seg.analogsignals)
    >>> print(seg.analogsignals[0])

To get a list of directly readable objects:

    >>> MyFormatIO.readable_objects
    [Segment]

The first element of the previous list is the highest level for reading the file. This mean that the IO has a `read_segment` method:

    >>> seg = reader.read_segment()
    >>> type(seg)
    neo.core.Segment


All IOs have a read() method that returns a list of `Block` objects (representing the whole content of the file):

    >>> bl = reader.read()
    >>> print bl[0].segments[0]
    neo.core.Segment


## Implemented IO functions

In [ ]:
dir(neo.io)